### Experimental:
Let me try Dube Fox Su (2009):
$$s_{jt}=\int_{0}^{\infty}\frac{exp\big(x'_{jt}\beta_{t}-\frac{1}{y_{i}}p_{jt}+\xi_{jt}\big)}{1+\sum_{k}exp\big(x'_{jt}\beta_{t}-\frac{1}{y_{i}}p_{jt}+\xi_{jt}\big)}\frac{1}{y_{i}\sigma_{y}\sqrt{2\pi}}\exp\Big(-\frac{(\log y_{i}-\mu_{y})^{2}}{2\sigma_{y}^{2}}\Big)dy_{i}=
$$
$$s_{jt}=\int_{-\infty}^{\infty}\frac{exp\big(x'_{jt}\beta_{t}-\frac{1}{\exp e_{i}}p_{jt}+\xi_{jt}\big)}{1+\sum_{k}exp\big(x'_{jt}\beta_{t}-\frac{1}{\exp e_{i}}p_{jt}+\xi_{jt}\big)}\frac{1}{\sigma_{y}\sqrt{2\pi}}\exp\Big(-\frac{(e_{i}-\mu_{y})^{2}}{2\sigma_{y}^{2}}\Big)de_{i}=
$$
$$s_{jt}=\int_{-\infty}^{\infty}\frac{1}{\sqrt\pi}\frac{exp\big(x'_{jt}\beta_{t}-\exp (-\mu_{y}-\sqrt{2}\sigma_{y}e_{i}) p_{jt}+\xi_{jt}\big)}{1+\sum_{k}exp\big(x'_{jt}\beta_{t}-\exp (-\mu_{y}-\sqrt{2}\sigma_{y}e_{i})p_{jt}+\xi_{jt}\big)}\phi(e_i)de_{i}=
$$

In [ ]:
year = 1990

w = convert(Array{Float64}, data[data[:year] .== year, [:share, :weight, :hp, :ac, :price]])
z = convert(Array{Float64}, data[data[:year] .== year, [:weight, :hp, :ac, :comp_weight, :comp_hp, :comp_ac,
                                                                      :firm_weight, :firm_hp, :firm_ac]]);

In [ ]:
using NLopt
using ForwardDiff

n = 5 + size(w)[1]

opt = Opt(:AUGLAG, n)

function f(ζ::Vector, grad::Vector)
    if length(grad) > 0
        grad[:] = 1/(size(ξ)^2) * vcat(zeros(5), 2 .* ζ[6:end]' * z * z')
    end
        
    return (1/size(ξ) .* z' * ζ[6:end])' * (1/size(ξ) .* z' * ζ[6:end])
end

min_objective!(opt, f)

function c(result::Vector, ζ::Vector, grad::Matrix)
    
    h(e, ζ) = 1/(√π) .* exp.(w[:, 2:4]' * ζ[1:3] .- exp(-ζ[4] -(√2) * ζ[5] * e) * w[:, 5] + ζ[6:end]) / 
            (1 + sum(exp.(w[:, 2:4]' * ζ[1:3] .- exp(-ζ[4] -(√2) * ζ[5] * e) * w[:, 5] + ζ[6:end])))
    
    (e, q) = gausshermite(30)
    
    h_wrapper(e) = return ζ -> -log.(hcat(h(e, ζ)...) * q)
    
    if length(grad) > 0
        ForwardDiff.jacobian(h_wrapper(e))(ζ)
    end
        
    result[:] = log.(w[:, 1]) - log.(hcat(h.(e)...) * q)
end

equality_constraint!(opt, c, 1e-6 * ones(n))
#maxtime!(opt, 60.)

optimize(opt, zeros(n))

In [ ]:
f(x) = x[1] * x[2]